# LW Profile and Statistics Plots for RRTMGP Band Generation

## Necessary Libraries

In [ ]:
# standard libraries
import os, sys
import datetime as DT

# pip install
import numpy as np
import matplotlib.pyplot as plot
from matplotlib import rc
import matplotlib.font_manager as font
import netCDF4 as nc
import time # for process_timeing processes

# for multi-page PDF files
from matplotlib.backends.backend_pdf import PdfPages

# Pernak Python module
LIBPATHS = ['common', 'RRTMGP-profile-stats-plots']
for path in LIBPATHS: sys.path.append(path)
import utils
from LBLRTM_RRTMGP_compare import *

## Global Variables

These variables will likely not to be changed for various runs of the `main()` code. So after this cell is run once, it will probably not need to be run again.

In [ ]:
# this changes with netCDF template version -- 
# older versions use 'band_wavenumbers', newer ones use 'band_lims_wvn'
# indexing for these arrays may also be reversed depending on version
BWVN = 'band_lims_wvn'

# default attributes to extract from input netCDFs
DEFATTR = ['dimensions', BWVN, 'emis_sfc', 'p_lay', \
  'p_lev', 't_lay', 't_lev', 't_sfc', 'vmr_h2o', 'vmr_co2', \
  'vmr_o3', 'vmr_ch4', 'vmr_co', 'vmr_n2o', 'vmr_o2', 'vmr_n2']

# trial and error spacing for subplots
HSPACE = 0.5

# forcing scenario; dimension 1 in RRTMGP netCDFs
IRECORD = 0

## main()

### Runtime Variables

These are variables that will likely need to be reassigned depending on the analysis to be done. We are using the default [plotting](https://github.com/AER-RC/RRTMGP-profile-stats-plots/blob/master/LBLRTM_RRTMGP_config.ini) [configurations](https://github.com/AER-RC/RRTMGP-profile-stats-plots/blob/master/LBLRTM_RRTMGP_compare.py#L1011) as the template for what inputs are passed to the plotting cell. If any of these variables are changed, this code cell will need to be re-run.

In [ ]:
# paths to temporary testing files
PROJECT = '/project/projectdirs/e3sm/pernak18/reference_netCDF'

# input files; NOTE: NO FORCING HAS BEEN IMPLEMENTED IN THIS NOTEBOOK
# reference LBL file will likely not change; test RRTMGP file will point 
# to netCDFs written with g-point-reduction Notebook
refDir = '{}/g-point-reduce'.format(PROJECT)
testDir = '{}/profile-stats-plots'.format(PROJECT)
refFile = '{}/lblrtm-lw-flux-inputs-outputs-garandANDpreind.nc'.format(refDir)
testFile = '{}/rrtmgp-lw-flux-inputs-outputs-garand-all.nc'.format(testDir) # reference 256-g-point RRTMGP
testFile = '/global/homes/p/pernak18/RRTMGP/g-point-reduction/optimized_fluxes_iter001.nc' # g-point combining solution

# make sure paths exist
paths = [refFile, testFile]
for path in paths: utils.file_check(path)

# plotting parameters
plot_profiles = True
plot_stats = True
xt = 'LBLRTM'
yt = 'RRTMGP-LBLRTM'
aType = 'Garand'
log_y = False
single_stat = True

# g-point reduction cost function, iteration number, and normalization ID
iCost = 1
iterID = 1
normID = 1
iterNorm = 'cost{:02d}_iter{:03d}_norm{:1d}'.format(iCost, iterID, normID)
iterNorm = os.path.basename(testFile)

# prefixes for output files
profPrefix = 'PROFS_LBLRTM_RRTMGP_{}'.format(iterNorm)
statPrefix = 'stats_lblrtm_rrtmgp_{}'.format(iterNorm)
#profPrefix = 'PROFS_LBLRTM_RRTMGP'
#statPrefix = 'stats_lblrtm_rrtmgp'

# pressure at tropopause (mbar)
pTrop = np.exp(4.6)

# unit-offset number of band to plot
# can also be "None" (all bands) or [] (no single bands)
bands = [4]

# convert to zero-offset band numbers; None ==> all bands
#inBand = None if bands is None else np.array(bands)-1
doBroadband = True

The following code cell should ONLY be run IF FORCING is being investigated.

In [ ]:
PIPPATH = '{}/.local/'.format(os.path.expanduser('~')) + \
    'cori/3.7-anaconda-2019.10/lib/python3.7/site-packages'

sys.path.append(PIPPATH)
import xarray as xa

iForce = 2

# newer RRTMGP-convention netCDFs for this project have forcing 
# scenarios consolidated into a single file, but the older plotting 
# programs expect separate files for each forcing scenario, so we 
# will create those files in this cell
with xa.open_dataset(refFile) as rDS, xa.open_dataset(testFile) as tDS:
    # baseline (no forcing) and forcing datasets
    allDS = [rDS.isel(record=0), tDS.isel(record=0), 
             rDS.isel(record=iForce), tDS.isel(record=iForce)]
# endwith

allNames = ['lblrtm-lw-flux-inputs-outputs-garand.nc', 
            'rrtmgp-lw-flux-inputs-outputs-garand.nc', 
            'lblrtm-lw-flux-inputs-outputs-garand-forcing.nc', 
            'rrtmgp-lw-flux-inputs-outputs-garand-forcing.nc']

for ds, name in zip(allDS, allNames):
    ds.attrs['model-description'] = ''
    ds.to_netcdf(name)
    print(name)
refFile, testFile = forcingDiff(allNames[0], allNames[1], allNames[2], allNames[3])


Now we'll proceed with profile plot generation. The process is time consuming, but the progress -- for netCDF reading, band number, column number -- is printed to standard output (i.e., in this notebook, after the code cell). Once the figures are saved, they can be opened in a separate Jupyter tab by clicking on them in the left sidebar (the names follow the convention `plotPrefix_??.pdf`, where `plotPrefix` was defined in the previous "Runtime Variables" code cell, and `??` is the zero-padded band number, which can also be `broadband` if `doBroadband` is set to `True`). Additionally, each page of the PDF is rendered in this notebook. Figures can be deleted from the **notebook** by right-clicking on the output and selecting the "Clear Outputs" pull down option.

In [ ]:
# plot the test and reference upward and downward flux together
# with the heating rates (and differences for each)
if plot_profiles:
    pBands = np.arange(16) if bands is None else np.array(bands)-1

    for iBand in pBands:
        profPDFs(refFile, testFile, yt, tPauseP=pTrop, \
          prefix=profPrefix, atmType=aType, inBand=iBand, \
          yLog=log_y)
    # end iBand loop

    # for specified bands AND broadband
    if doBroadband:
        profPDFs(refFile, testFile, yt, tPauseP=pTrop, \
          prefix=profPrefix, atmType=aType, \
          yLog=log_y, broadOnly=doBroadband)
    # end broadband plot
# end plot_profiles

A similar procedure is performed for the statistics plots, which are less time consuming to produce. Because of the smaller amount of time needed for the statistics, they are calculated for all bands and broadband. Again, status is written to standard output, and files can be accessed in the left sidebar. File names conform to the `statPrefix_??.pdf` convention, where `statPrefix` is defined in the "Runtime Variables" code cell, and `??` is the zero-padded band number or `broadband`. Plots are also rendered in standard output. Figures can be deleted from the **notebook** by right-clicking on the output and selecting the "Clear Outputs" pull down option.

In [ ]:
# plot profile statistics
if plot_stats:
    statPDF(refFile, testFile, singlePDF=single_stat, \
      tPauseP=pTrop, xTitle=xt, yTitle=yt, prefix=statPrefix, \
      atmType=aType, statCSV='', forcing=False)